### The purpose of this notebook is to build collaborative filtering method for recommendations

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from datetime import datetime, timedelta
import collections

from util_transaction_data import *

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)


In [2]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.preprocessing import *
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpmax

### first, prepare the data
 - will remove the "system account", i.e., merchant 191 and 11, also remove cigarette catgories
 - we will use "top_cat" as items
 - we will use the category names instead of category IDs
 - need to download the category name data file from https://drive.google.com/file/d/1PmuZ9XsTWWi4tA_8_qaqSFscYVZPX7LJ/view?usp=share_link
 

In [3]:
filename = './AwanTunai_transaction_data.csv'
baskets = pd.read_csv(filename)

baskets = baskets[baskets.merchant_id !=191] # this is a system account
baskets = baskets[baskets.merchant_id !=11] # this is a system account
baskets = baskets[baskets.top_cat !=27] # remove cgarette top categories
baskets = baskets[baskets.sub_cat !=86] # remove cigarette sub categories

baskets.dropna(inplace=True)

cat_mapping_file = './top_category_mapping_new.csv'
top_cat_names = pd.read_csv(cat_mapping_file)
df = baskets.merge(top_cat_names[["top_cat_id","top_cat_en"]], left_on="top_cat",right_on="top_cat_id")
baskets = df.drop(axis=1, columns=["top_cat_id"])

In [4]:
transaction_data = []
for groups in baskets.groupby('order_id').groups.values():
    transaction_data.append(list(baskets.loc[groups]['top_cat_en'].to_numpy()))
transaction_data[:10]

[['general medicine', 'instant food', 'instant food'],
 ['general medicine', 'powder drink'],
 ['sweetend condensed milk', 'packaged drink'],
 ['general medicine', 'instant food'],
 ['instant food'],
 ['general medicine'],
 ['general medicine', 'powder drink', 'packaged drink'],
 ['general medicine', 'powder drink'],
 ['sweetend condensed milk'],
 ['powder drink']]

In [5]:
def oneHotCoding(transaction_data):
    te = TransactionEncoder()
    te_ary = te.fit(transaction_data).transform(transaction_data)
    te_df = pd.DataFrame(te_ary, columns=te.columns_)
    return te_df

transaction_oneHot = oneHotCoding(transaction_data)

In [6]:
transaction_oneHot

,baby care,baby food,bath soap,body care,bottled water,breakfast food,canned food,carpentry tools,cream soap,dental care,dish soap,external medicine,floor washing soap,food materials,gas lighter,general medicine,hair care,household general supplis,household hygiene supplies,ingredients,instant food,laundry soap,medical supplies,packaged drink,packaged tea,playing cards,powder drink,sachet drink,snack,stationary,sweetend condensed milk,syrup,toys,womens hygiene supplies
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23955,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False,True,False,False,False,False,True,False,False,False,False,False
23956,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False,True,False,False,False,False,False
23957,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,True,False,True,True,True,False,False,False,False,False
23958,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False


In [7]:
transaction_oneHot.sum()

baby care                      1771
baby food                       493
bath soap                      3102
body care                      2926
bottled water                   119
breakfast food                   99
canned food                      31
carpentry tools                 168
cream soap                      182
dental care                    2260
dish soap                       400
external medicine              2195
floor washing soap              797
food materials                 1075
gas lighter                       8
general medicine               9489
hair care                      2848
household general supplis      2947
household hygiene supplies     3152
ingredients                   10910
instant food                   9294
laundry soap                   4798
medical supplies                374
packaged drink                 9104
packaged tea                   1917
playing cards                     8
powder drink                   7869
sachet drink                

### build user-item interaction matrix
 - using qty as degree of "like", and take sum 
 - fillin NA with zero for not purchased before

In [8]:
merchant_cat = baskets.pivot_table(values='qty', index=["merchant_id"], columns="top_cat_en", aggfunc=np.sum).reset_index()
merchant_cat.set_index(keys="merchant_id",inplace=True)
merchant_cat.fillna(value=0,inplace = True)

In [9]:
merchant_cat.corrwith(merchant_cat["baby food"],axis=0)

top_cat_en
baby care                     0.396975
baby food                     1.000000
bath soap                     0.256129
body care                     0.554916
bottled water                -0.032959
breakfast food                0.057388
canned food                   0.034068
carpentry tools               0.211680
cream soap                    0.213045
dental care                   0.450265
dish soap                     0.273086
external medicine             0.500080
floor washing soap            0.431750
food materials                0.295532
gas lighter                  -0.024515
general medicine              0.406682
hair care                     0.539239
household general supplis     0.397212
household hygiene supplies    0.351180
ingredients                   0.405955
instant food                  0.288383
laundry soap                  0.475457
medical supplies              0.195857
packaged drink                0.351627
packaged tea                  0.432563
playing cards 

In [10]:
merchant_cat.corrwith(merchant_cat["baby food"]).argmin()

4

In [11]:
cat_cat_corr = pd.DataFrame(np.zeros(merchant_cat.shape[1]**2).reshape([merchant_cat.shape[1],merchant_cat.shape[1]]), columns = merchant_cat.columns, index=merchant_cat.columns)
for i in merchant_cat.columns:
    cat_cat_corr[i]= merchant_cat.corrwith(merchant_cat[i])
cat_cat_corr

top_cat_en,baby care,baby food,bath soap,body care,bottled water,breakfast food,canned food,carpentry tools,cream soap,dental care,dish soap,external medicine,floor washing soap,food materials,gas lighter,general medicine,hair care,household general supplis,household hygiene supplies,ingredients,instant food,laundry soap,medical supplies,packaged drink,packaged tea,playing cards,powder drink,sachet drink,snack,stationary,sweetend condensed milk,syrup,toys,womens hygiene supplies
top_cat_en,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
baby care,1.000000,0.396975,0.429120,0.634286,-0.028287,0.121747,0.038023,0.254697,0.280109,0.506486,0.394041,0.481889,0.333864,0.276506,-0.012051,0.561693,0.675722,0.461543,0.597172,0.471210,0.369565,0.635651,0.366956,0.445339,0.286175,0.245426,0.246930,0.534393,0.607664,0.240415,0.620062,0.436453,0.292105,0.632554
baby food,0.396975,1.000000,0.256129,0.554916,-0.032959,0.057388,0.034068,0.211680,0.213045,0.450265,0.273086,0.500080,0.431750,0.295532,-0.024515,0.406682,0.539239,0.397212,0.351180,0.405955,0.288383,0.475457,0.195857,0.351627,0.432563,0.028049,0.163759,0.474200,0.472162,0.220575,0.490765,0.486140,0.081129,0.463957
bath soap,0.429120,0.256129,1.000000,0.586018,0.315250,0.146864,0.271885,0.382762,0.339904,0.550965,0.502567,0.356403,0.486710,0.236440,0.381966,0.536326,0.443500,0.608718,0.418073,0.572721,0.446195,0.610728,0.281052,0.495931,0.448329,0.125664,0.439868,0.608294,0.600862,0.464209,0.652132,0.166001,0.097428,0.357376
body care,0.634286,0.554916,0.586018,1.000000,0.034469,0.077678,0.095329,0.299216,0.418053,0.715938,0.502836,0.581634,0.501509,0.392706,0.056159,0.662165,0.785091,0.661178,0.659819,0.693779,0.563526,0.752631,0.316498,0.601675,0.511609,0.125190,0.493223,0.821326,0.782928,0.521343,0.704383,0.458364,0.327380,0.599876
bottled water,-0.028287,-0.032959,0.315250,0.034469,1.000000,-0.016208,0.532108,-0.011692,-0.014162,0.008883,-0.022018,0.051882,-0.007415,-0.009434,0.815594,0.329878,0.009537,-0.007316,0.144469,0.098299,0.031553,0.056648,-0.008572,0.148457,0.015465,-0.011033,0.352462,0.115301,0.012034,0.227201,0.040603,-0.038009,-0.018108,-0.029099
breakfast food,0.121747,0.057388,0.146864,0.077678,-0.016208,1.000000,-0.010645,0.161220,0.167128,0.084663,0.004858,0.094138,0.055343,0.094568,-0.010651,0.059587,0.084509,0.125780,0.151087,0.126466,0.070119,0.133741,0.101196,0.072014,0.080583,-0.029637,0.022489,0.124043,0.104000,0.059091,0.093816,0.176451,-0.033631,0.063098
canned food,0.038023,0.034068,0.271885,0.095329,0.532108,-0.010645,1.000000,-0.007687,0.022882,0.135017,0.069147,0.180663,0.037071,0.102421,0.656921,0.317651,0.038012,0.010351,0.225363,0.196452,0.052761,0.134321,0.109081,0.326119,0.053204,-0.008975,0.299692,0.193493,0.083400,0.182632,0.096834,-0.020562,-0.008580,0.078420
carpentry tools,0.254697,0.211680,0.382762,0.299216,-0.011692,0.161220,-0.007687,1.000000,0.163430,0.534703,0.198858,0.388714,0.373845,0.245213,-0.012212,0.303479,0.175515,0.574169,0.232623,0.391280,0.329444,0.429717,0.157089,0.276287,0.102037,-0.018428,0.409426,0.375094,0.377912,0.406239,0.402863,0.135589,-0.016850,0.152312
cream soap,0.280109,0.213045,0.339904,0.418053,-0.014162,0.167128,0.022882,0.163430,1.000000,0.387082,0.355452,0.300910,0.300514,0.278256,-0.009027,0.230772,0.381010,0.355522,0.269526,0.388788,0.276017,0.441454,0.157689,0.296533,0.219419,0.132559,0.137046,0.370120,0.322394,0.229128,0.373508,0.233154,0.062268,0.270975


In [12]:
def cosine_similarity(v1, v2):
    return np.dot(v1,v2) / math.sqrt(np.dot(v1,v1) * np.dot(v2,v2))
cosine_similarity(v1=[1, 20.0, 9.0], v2=[4, 5.0, 6])

0.8201403991329932

In [13]:
cosine_similarity(merchant_cat["baby food"], merchant_cat["body care"])

0.643400424822213

In [14]:
cat_cat_cos = pd.DataFrame(np.zeros(merchant_cat.shape[1]**2).reshape([merchant_cat.shape[1],merchant_cat.shape[1]]), columns = merchant_cat.columns, index=merchant_cat.columns)
for i in merchant_cat.columns:
    for j in merchant_cat.columns:
        cat_cat_cos[i][j] = cosine_similarity(merchant_cat[i],merchant_cat[j])
cat_cat_cos

top_cat_en,baby care,baby food,bath soap,body care,bottled water,breakfast food,canned food,carpentry tools,cream soap,dental care,dish soap,external medicine,floor washing soap,food materials,gas lighter,general medicine,hair care,household general supplis,household hygiene supplies,ingredients,instant food,laundry soap,medical supplies,packaged drink,packaged tea,playing cards,powder drink,sachet drink,snack,stationary,sweetend condensed milk,syrup,toys,womens hygiene supplies
top_cat_en,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
baby care,1.000000,0.518093,0.518121,0.720958,0.023034,0.202164,0.085742,0.346284,0.362601,0.606885,0.486870,0.600046,0.445397,0.356043,0.016845,0.672577,0.752105,0.582102,0.692899,0.592674,0.481504,0.729180,0.490131,0.566703,0.405192,0.281708,0.389700,0.655856,0.698843,0.360301,0.719310,0.559574,0.361997,0.718333
baby food,0.518093,1.000000,0.360655,0.643400,0.010896,0.134911,0.075877,0.298256,0.295087,0.547127,0.372953,0.596808,0.514730,0.364641,0.001135,0.532664,0.630524,0.512339,0.481890,0.521924,0.398917,0.587562,0.331325,0.473060,0.514558,0.084723,0.300552,0.587768,0.576005,0.327876,0.600893,0.582100,0.169163,0.569839
bath soap,0.518121,0.360655,1.000000,0.646950,0.328282,0.203722,0.290990,0.438741,0.396921,0.613369,0.558978,0.455469,0.548580,0.300618,0.377720,0.607723,0.529539,0.664754,0.509128,0.635347,0.516351,0.667264,0.379300,0.569603,0.514251,0.165538,0.514020,0.665138,0.658874,0.524933,0.699644,0.292447,0.168662,0.457994
body care,0.720958,0.643400,0.646950,1.000000,0.077526,0.164359,0.135457,0.383630,0.478925,0.772586,0.576399,0.676880,0.582224,0.454247,0.076302,0.747314,0.835642,0.736779,0.740547,0.763967,0.638549,0.815582,0.449670,0.688489,0.589451,0.177731,0.587403,0.866666,0.833285,0.591577,0.780983,0.576516,0.391911,0.693163
bottled water,0.023034,0.010896,0.328282,0.077526,1.000000,0.003867,0.537029,0.015327,0.011741,0.049365,0.011937,0.091417,0.027771,0.015453,0.815896,0.331111,0.055575,0.038542,0.173194,0.132375,0.065678,0.099725,0.031737,0.175942,0.048765,0.003092,0.360972,0.149365,0.057177,0.245763,0.087043,0.010194,0.005349,0.021440
breakfast food,0.202164,0.134911,0.203722,0.164359,0.003867,1.000000,0.011412,0.206597,0.210291,0.160263,0.071577,0.175738,0.122048,0.140178,0.000880,0.154707,0.169749,0.202007,0.227340,0.204151,0.138941,0.217937,0.172039,0.153629,0.144613,0.000000,0.099305,0.210910,0.185547,0.121624,0.186483,0.244880,0.015045,0.150688
canned food,0.085742,0.075877,0.290990,0.135457,0.537029,0.011412,1.000000,0.022019,0.050043,0.167414,0.100900,0.209384,0.072849,0.125733,0.658108,0.325903,0.085418,0.058971,0.248377,0.223264,0.089124,0.170646,0.142956,0.338200,0.087560,0.006628,0.316501,0.220351,0.124518,0.207253,0.139634,0.030441,0.017032,0.120305
carpentry tools,0.346284,0.298256,0.438741,0.383630,0.015327,0.206597,0.022019,1.000000,0.222298,0.581121,0.270931,0.457670,0.432673,0.296581,0.003502,0.390735,0.279088,0.615509,0.327825,0.460081,0.395557,0.494573,0.248041,0.359668,0.187319,0.021199,0.468992,0.450395,0.449147,0.459080,0.472966,0.238635,0.047724,0.258737
cream soap,0.362601,0.295087,0.396921,0.478925,0.011741,0.210291,0.050043,0.222298,1.000000,0.448459,0.410125,0.378263,0.363352,0.325240,0.005858,0.325409,0.447506,0.424212,0.353750,0.453432,0.344220,0.498989,0.244187,0.372528,0.289546,0.163749,0.223611,0.440910,0.397395,0.295070,0.443758,0.317783,0.119150,0.354000


In [15]:
merchant_cat.columns[ merchant_cat.corrwith(merchant_cat["baby food"]).argmin()]

'bottled water'

In [16]:
cat_cat_cos.idxmin(axis=1)

top_cat_en
baby care                        gas lighter
baby food                        gas lighter
bath soap                      playing cards
body care                        gas lighter
bottled water                  playing cards
breakfast food                 playing cards
canned food                    playing cards
carpentry tools                  gas lighter
cream soap                       gas lighter
dental care                      gas lighter
dish soap                        gas lighter
external medicine              bottled water
floor washing soap               gas lighter
food materials                   gas lighter
gas lighter                    playing cards
general medicine              breakfast food
hair care                        gas lighter
household general supplis        gas lighter
household hygiene supplies     bottled water
ingredients                    bottled water
instant food                   bottled water
laundry soap                     gas lighter

### need to drop "gas lighter" category

In [17]:
cat_cat_cos["baby food"].nlargest()

top_cat_en
baby food                  1.000000
body care                  0.643400
hair care                  0.630524
sweetend condensed milk    0.600893
external medicine          0.596808
Name: baby food, dtype: float64

 - observe the result difference between correlation and consine similarity

In [18]:
cat_cat_corr["baby food"].nlargest()

top_cat_en
baby food                  1.000000
body care                  0.554916
hair care                  0.539239
external medicine          0.500080
sweetend condensed milk    0.490765
Name: baby food, dtype: float64

In [19]:
merchant_id = merchant_cat.index.values
cat_name = merchant_cat.columns.values
merchant_id[348], cat_name[4]

(357, 'bottled water')

In [20]:
merchant_cat.loc[357]

top_cat_en
baby care                      1.0
baby food                      0.0
bath soap                      0.0
body care                      0.0
bottled water                  0.0
breakfast food                 0.0
canned food                    0.0
carpentry tools                0.0
cream soap                     0.0
dental care                    0.0
dish soap                      0.0
external medicine              0.0
floor washing soap             0.0
food materials                 0.0
gas lighter                    0.0
general medicine               0.0
hair care                      0.0
household general supplis      0.0
household hygiene supplies    10.0
ingredients                    3.0
instant food                   2.0
laundry soap                   0.0
medical supplies               0.0
packaged drink                11.0
packaged tea                   0.0
playing cards                  0.0
powder drink                   5.0
sachet drink                   4.0
snack    

In [21]:
type(merchant_cat.loc[357]), type(merchant_cat[merchant_cat.index == 357])

(pandas.core.series.Series, pandas.core.frame.DataFrame)

In [22]:
merchant_cat[merchant_cat.index == 357].equals(merchant_cat.loc[357])

False

In [23]:
merchant_cat[merchant_cat.index == 357].eq(merchant_cat.loc[357])

top_cat_en,baby care,baby food,bath soap,body care,bottled water,breakfast food,canned food,carpentry tools,cream soap,dental care,dish soap,external medicine,floor washing soap,food materials,gas lighter,general medicine,hair care,household general supplis,household hygiene supplies,ingredients,instant food,laundry soap,medical supplies,packaged drink,packaged tea,playing cards,powder drink,sachet drink,snack,stationary,sweetend condensed milk,syrup,toys,womens hygiene supplies
merchant_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
357,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


In [24]:
merchant_cat.loc[357] == merchant_cat[merchant_cat.index == 357]

top_cat_en,baby care,baby food,bath soap,body care,bottled water,breakfast food,canned food,carpentry tools,cream soap,dental care,dish soap,external medicine,floor washing soap,food materials,gas lighter,general medicine,hair care,household general supplis,household hygiene supplies,ingredients,instant food,laundry soap,medical supplies,packaged drink,packaged tea,playing cards,powder drink,sachet drink,snack,stationary,sweetend condensed milk,syrup,toys,womens hygiene supplies
merchant_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
357,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


In [25]:
merchant_cat[merchant_cat.index == 357].values , merchant_cat.loc[357].values

(array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0., 10.,  3.,  2.,  0.,  0., 11.,  0.,  0.,
          5.,  4.,  6.,  0.,  0.,  0.,  0.,  0.]]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0., 10.,  3.,  2.,  0.,  0., 11.,  0.,  0.,
         5.,  4.,  6.,  0.,  0.,  0.,  0.,  0.]))

In [26]:
cosine_similarity(merchant_cat.loc[357],merchant_cat.loc[1])

0.5798335076138013

In [27]:
mer_mer_cos = pd.DataFrame(np.zeros(merchant_cat.shape[0]**2).reshape([merchant_cat.shape[0],merchant_cat.shape[0]]), columns = merchant_cat.index, index=merchant_cat.index)
for i in merchant_cat.index:
    for j in merchant_cat.index:
        mer_mer_cos[i][j] = cosine_similarity(merchant_cat[merchant_cat.index == i].values.ravel(), merchant_cat[merchant_cat.index == j].values.ravel())
        
mer_mer_cos

merchant_id,1,2,3,4,5,6,8,9,10,12,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357
merchant_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.692956,0.725940,0.440529,0.419919,0.415271,0.393919,0.416954,0.808246,0.626111,0.583812,0.734804,0.617319,0.805154,0.495931,0.700518,0.669211,0.811533,0.360663,0.357226,0.787162,0.221623,0.670880,0.818059,0.494046,0.691383,0.902828,0.504316,0.561257,0.792528,0.423518,0.317345,0.881196,0.676599,0.759139,0.701545,0.845463,0.842809,0.236638,0.885740,0.817827,0.660825,0.397059,0.609156,0.759982,0.764676,0.874455,0.571268,0.724176,0.744346,0.657658,0.737591,0.829150,0.814192,0.579351,0.881978,0.595658,0.636185,0.679084,0.761820,0.725759,0.716486,0.831455,0.551052,0.861532,0.132348,0.583857,0.622140,0.696066,0.432066,0.828584,0.438905,0.695699,0.365906,0.690708,0.610863,0.812653,0.611297,0.664870,0.452778,0.649934,0.756424,0.730725,0.756503,0.848751,0.800265,0.820252,0.472048,0.593690,0.865878,0.647027,0.883101,0.623057,0.870348,0.604843,0.776003,0.832075,0.769806,0.744312,0.778986,0.587555,0.318297,0.693091,0.597630,0.622403,0.584606,0.879234,0.405599,0.818444,0.752714,0.655000,0.830745,0.852698,0.709822,0.612514,0.738776,0.713292,0.552391,0.271037,0.806473,0.845123,0.759225,0.607162,0.726756,0.790110,0.758001,0.644261,0.693248,0.742403,0.764922,0.652380,0.528698,0.661122,0.372441,0.525835,0.254856,0.709873,0.302013,0.612915,0.541674,0.378804,0.748390,0.609735,0.383589,0.924470,0.621020,0.833397,0.710605,0.695128,0.015184,0.400569,0.607702,0.799427,0.227479,0.765647,0.807887,0.718588,0.357883,0.798682,0.328883,0.737974,0.536282,0.631337,0.536821,0.564513,0.769105,0.686668,0.545065,0.817912,0.512856,0.786920,0.759323,0.884302,0.668288,0.806842,0.604329,0.795003,0.555824,0.420857,0.629889,0.562376,0.459573,0.441590,0.394894,0.409441,0.755991,0.658574,0.696878,0.874083,0.841051,0.760478,0.773529,0.832257,0.485183,0.700719,0.802366,0.901576,0.670514,0.779491,0.749217,0.768707,0.732114,0.800804,0.573776,0.591351,0.780118,0.673013,0.777282,0.703546,0.590879,0.659543,0.765006,0.628376,0.779965,0.717397,0.872259,0.852585,0.601344,0.676223,0.815232,0.751364,0.027580,0.821076,0.586282,0.655871,0.741568,0.815535,0.762683,0.654039,0.800464,0.773236,0.867665,0.752663,0.855828,0.675303,0.489350,0.840897,0.662045,0.688988,0.768406,0.692303,0.708154,0.785415,0.597437,0.836481,0.664710,0.525870,0.750749,0.797215,0.385532,0.653931,0.622386,0.618369,0.346114,0.817765,0.779082,0.660586,0.368478,0.830278,0.

In [28]:
mer_mer_corr = pd.DataFrame(np.zeros(merchant_cat.shape[0]**2).reshape([merchant_cat.shape[0],merchant_cat.shape[0]]), columns = merchant_cat.index, index=merchant_cat.index)
for i in merchant_cat.index:
    mer_mer_corr.loc[i]= merchant_cat.corrwith(merchant_cat.loc[i],axis=1)
mer_mer_corr


merchant_id,1,2,3,4,5,6,8,9,10,12,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357
merchant_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.625554,0.640383,0.363658,0.307148,0.329479,0.360023,0.294416,0.759395,0.578079,0.467288,0.658658,0.504089,0.748314,0.383369,0.632920,0.570467,0.758163,0.217538,0.280461,0.746648,0.092433,0.629760,0.766075,0.437395,0.646563,0.877719,0.430140,0.436955,0.741159,0.257640,0.251926,0.856252,0.573686,0.684457,0.604017,0.801403,0.800134,0.114414,0.854669,0.766953,0.577860,0.229701,0.504161,0.685638,0.692348,0.843782,0.450737,0.642192,0.667894,0.555466,0.661822,0.780510,0.759707,0.475929,0.849333,0.463355,0.511799,0.578856,0.690430,0.637478,0.626754,0.787211,0.409146,0.822664,0.023092,0.512482,0.510035,0.596850,0.319365,0.780762,0.365550,0.604685,0.259164,0.589569,0.479724,0.757794,0.505932,0.572679,0.399978,0.624236,0.680674,0.645493,0.684659,0.806575,0.755439,0.770291,0.339394,0.455278,0.829950,0.534790,0.850910,0.494692,0.836823,0.455825,0.707840,0.784226,0.699274,0.665313,0.713118,0.488236,1.316221e-01,0.641203,0.498031,0.504344,0.484855,0.845768,0.218976,0.765478,0.677560,0.599083,0.782817,0.811040,0.631078,0.504759,0.655922,0.625545,0.395556,0.155422,0.751708,0.800949,0.687611,0.479774,0.643345,0.727133,0.682872,0.542602,0.597004,0.664601,0.692485,0.554771,0.400675,0.562241,0.320337,0.379958,0.126671,0.616910,0.165205,0.482632,0.413822,0.222377,0.669655,0.514455,0.268036,0.904283,0.496031,0.792340,0.636635,0.595804,-0.077762,0.322683,0.483683,0.746099,0.105990,0.705761,0.751712,0.660473,0.210011,0.738786,0.178816,0.657134,0.399097,0.526348,0.420456,0.477379,0.698386,0.621368,0.440455,0.764932,0.358804,0.733538,0.686301,0.852393,0.590568,0.749816,0.471914,0.734122,0.422461,0.308437,0.553981,0.505832,0.434662,0.337261,0.301574,0.339400,0.681904,0.562580,0.621420,0.839440,0.796283,0.687707,0.705833,0.793084,0.412181,0.637347,0.748644,0.876746,0.586831,0.712677,0.680500,0.749456,0.667786,0.741752,0.532219,0.500368,0.724530,0.574123,0.759705,0.610842,0.583940,0.551216,0.692709,0.518788,0.715295,0.639987,0.836937,0.813616,0.462042,0.571521,0.761353,0.673619,-0.133249,0.779931,0.453206,0.538604,0.688065,0.761507,0.690936,0.577378,0.742266,0.714542,0.831266,0.675659,0.817297,0.657944,0.355874,0.796205,0.585763,0.595599,0.700440,0.605473,0.662680,0.753948,0.489996,0.795881,0.628366,0.377956,0.691268,0.758553,0.284451,0.547394,0.486125,0.500688,0.220818,0.764580,0.714757,0.574376,0.244789,0.781

In [29]:
mer_mer_cos.loc[357].nlargest(n=10)

merchant_id
357    1.000000
192    0.944425
300    0.819988
38     0.791285
242    0.789093
299    0.739261
219    0.732227
200    0.732160
295    0.724406
140    0.723780
Name: 357, dtype: float64

 - notice the difference in ranking again

In [30]:
mer_mer_corr.loc[357].nlargest(n=10)

merchant_id
357    1.000000
192    0.935514
300    0.782023
38     0.747205
242    0.741917
299    0.692355
200    0.680775
219    0.666765
295    0.662408
158    0.656225
Name: 357, dtype: float64

In [31]:
from scipy import spatial
mer_cat_357=merchant_cat.loc[357]
mer_cat_192=merchant_cat.loc[192]

1 - spatial.distance.cosine(mer_cat_357,mer_cat_192)

0.9444248405749003

 - spot checking: our cosine is close enough to spatial.distance.cosine 

In [32]:
cosine_similarity(mer_cat_357,mer_cat_192)

0.9444248405749004

In [33]:
merchant_cat[(merchant_cat.index ==357) | (merchant_cat.index ==192)]

top_cat_en,baby care,baby food,bath soap,body care,bottled water,breakfast food,canned food,carpentry tools,cream soap,dental care,dish soap,external medicine,floor washing soap,food materials,gas lighter,general medicine,hair care,household general supplis,household hygiene supplies,ingredients,instant food,laundry soap,medical supplies,packaged drink,packaged tea,playing cards,powder drink,sachet drink,snack,stationary,sweetend condensed milk,syrup,toys,womens hygiene supplies
merchant_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
192,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,1.0,15.0,9.0,10.0,1.0,0.0,21.0,0.0,0.0,16.0,9.0,10.0,0.0,0.0,0.0,0.0,3.0
357,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,3.0,2.0,0.0,0.0,11.0,0.0,0.0,5.0,4.0,6.0,0.0,0.0,0.0,0.0,0.0


In [34]:
def most_similar(merchant_id, n=10):
    return mer_mer_cos.loc[merchant_id].nlargest(n=n).drop([merchant_id]).index.values
most_similar(357)

array([192, 300,  38, 242, 299, 219, 200, 295, 140])

 - get the highest ranked categories purchased by the closest merchants

In [35]:
merchant_cat.loc[most_similar(357, n=10)].mean().nlargest(n=7).index.values

array(['powder drink', 'snack', 'household hygiene supplies',
       'general medicine', 'packaged drink', 'ingredients',
       'instant food'], dtype=object)

 - get the categories purchased by the merchant

In [36]:
merchant_cat.columns[merchant_cat.loc[357].values.nonzero()].values

array(['baby care', 'household hygiene supplies', 'ingredients',
       'instant food', 'packaged drink', 'powder drink', 'sachet drink',
       'snack'], dtype=object)

 - take the merchant's categories out the closest neighbors' top categories

In [37]:
set(merchant_cat.loc[most_similar(357)].mean().nlargest().index.values) \
    - set(merchant_cat.columns[merchant_cat.loc[357].values.nonzero()].values)

{'general medicine'}

 - or take those categories that the merchant did not buy, value.eq(0)

In [38]:
merchant_cat.loc[most_similar(357, n=10)].mean()[merchant_cat.loc[357].eq(0)].sort_values(ascending=False)

top_cat_en
general medicine             62.111111
sweetend condensed milk      21.777778
hair care                    19.000000
bath soap                    16.444444
body care                    10.888889
laundry soap                  9.555556
external medicine             9.444444
breakfast food                5.333333
household general supplis     5.000000
dental care                   3.555556
packaged tea                  3.444444
womens hygiene supplies       3.000000
syrup                         2.666667
medical supplies              2.444444
carpentry tools               2.000000
food materials                2.000000
baby food                     1.222222
floor washing soap            0.444444
stationary                    0.222222
bottled water                 0.111111
dish soap                     0.000000
cream soap                    0.000000
playing cards                 0.000000
canned food                   0.000000
toys                          0.000000
gas lighter   

 - get the categories that the merchant bought

In [39]:
items = [col for i , col in enumerate(merchant_cat[merchant_cat.index==357]) if merchant_cat.loc[357][col]>0]
items

['baby care',
 'household hygiene supplies',
 'ingredients',
 'instant food',
 'packaged drink',
 'powder drink',
 'sachet drink',
 'snack']

In [40]:
merchant_cat.loc[357,items]

top_cat_en
baby care                      1.0
household hygiene supplies    10.0
ingredients                    3.0
instant food                   2.0
packaged drink                11.0
powder drink                   5.0
sachet drink                   4.0
snack                          6.0
Name: 357, dtype: float64

In [41]:
merchant_cat.columns[merchant_cat[merchant_cat.index==357].values.nonzero()[1]]

Index(['baby care', 'household hygiene supplies', 'ingredients',
       'instant food', 'packaged drink', 'powder drink', 'sachet drink',
       'snack'],
      dtype='object', name='top_cat_en')

In [42]:
merchant_cat.columns[merchant_cat.loc[357].gt(0)]

Index(['baby care', 'household hygiene supplies', 'ingredients',
       'instant food', 'packaged drink', 'powder drink', 'sachet drink',
       'snack'],
      dtype='object', name='top_cat_en')

In [43]:
merchant_cat[merchant_cat.columns[merchant_cat.loc[357].eq(0)]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 349 entries, 1 to 357
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   baby food                  349 non-null    float64
 1   bath soap                  349 non-null    float64
 2   body care                  349 non-null    float64
 3   bottled water              349 non-null    float64
 4   breakfast food             349 non-null    float64
 5   canned food                349 non-null    float64
 6   carpentry tools            349 non-null    float64
 7   cream soap                 349 non-null    float64
 8   dental care                349 non-null    float64
 9   dish soap                  349 non-null    float64
 10  external medicine          349 non-null    float64
 11  floor washing soap         349 non-null    float64
 12  food materials             349 non-null    float64
 13  gas lighter                349 non-null    float64

In [44]:
merchant_cat[merchant_cat.columns[merchant_cat.loc[357].gt(0)]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 349 entries, 1 to 357
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   baby care                   349 non-null    float64
 1   household hygiene supplies  349 non-null    float64
 2   ingredients                 349 non-null    float64
 3   instant food                349 non-null    float64
 4   packaged drink              349 non-null    float64
 5   powder drink                349 non-null    float64
 6   sachet drink                349 non-null    float64
 7   snack                       349 non-null    float64
dtypes: float64(8)
memory usage: 32.6 KB


### putting all pieces together
 - given a merchant ID
 - given the number of merchant neighbors to be considered for similarity
 - given the number of categories to recommend to the given merchant

In [45]:
def get_merchant_cat_IM(baskets):
    merchant_cat = baskets.pivot_table(values='qty', index=["merchant_id"], columns="top_cat_en", aggfunc=np.sum)
    merchant_cat.fillna(value=0,inplace = True)
    return merchant_cat

def get_merchant_similarity_matrix(baskets):
    merchant_cat = get_merchant_cat_IM(baskets)
    for i in merchant_cat.index:
        for j in merchant_cat.index:
            mer_mer_cos[i][j] = cosine_similarity(merchant_cat[merchant_cat.index == i].values.ravel(), merchant_cat[merchant_cat.index == j].values.ravel())
    return mer_mer_cos

def get_most_similar(merchant_similarity_matrix, merchant_id, n=10):
    return merchant_similarity_matrix.loc[merchant_id].nlargest(n=n+1).drop([merchant_id]).index.values

def get_recommended_categories(merchant_similarity_matrix, m_id, n_neighbor, n_cats):
    merchant_cat = get_merchant_cat_IM(baskets)
    neighbors = get_most_similar(merchant_similarity_matrix, merchant_id = m_id, n = n_neighbor)
    #rec_set = set(merchant_cat.loc[neighbors].mean().nlargest(n=n_cats).index.values)
    #merchant_set = set(merchant_cat.columns[merchant_cat.loc[357].values.nonzero()].values)
    #return rec_set - merchant_set
    rec_set = merchant_cat.loc[neighbors].mean()[merchant_cat.loc[m_id].eq(0)].sort_values(ascending=False).index.values[:n_cats]
    return rec_set

mer_mer_cos = get_merchant_similarity_matrix(baskets)
get_recommended_categories(mer_mer_cos,357,10,100)

array(['general medicine', 'sweetend condensed milk', 'hair care',
       'bath soap', 'body care', 'laundry soap', 'external medicine',
       'breakfast food', 'household general supplis', 'dental care',
       'packaged tea', 'womens hygiene supplies', 'syrup',
       'medical supplies', 'carpentry tools', 'food materials',
       'baby food', 'floor washing soap', 'stationary', 'bottled water',
       'dish soap', 'cream soap', 'playing cards', 'canned food', 'toys',
       'gas lighter'], dtype=object)

In [55]:
get_recommended_categories(mer_mer_cos,357,10,7)

array(['general medicine', 'sweetend condensed milk', 'hair care',
       'bath soap', 'body care', 'laundry soap', 'external medicine'],
      dtype=object)

In [46]:
def get_recommended_categories_set(merchant_similarity_matrix, m_id, n_neighbor, n_cats):
    merchant_cat = get_merchant_cat_IM(baskets)
    neighbors = get_most_similar(merchant_similarity_matrix, merchant_id = m_id, n = n_neighbor)
    rec_set = set(merchant_cat.loc[neighbors].mean().nlargest(n=n_cats).index.values)
    merchant_set = set(merchant_cat.columns[merchant_cat.loc[357].values.nonzero()].values)
    return rec_set - merchant_set
get_recommended_categories_set(mer_mer_cos,357,10,10)

{'general medicine', 'hair care', 'sweetend condensed milk'}

In [47]:
get_recommended_categories(mer_mer_cos,357,10,10)

array(['general medicine', 'sweetend condensed milk', 'hair care',
       'bath soap', 'body care', 'laundry soap', 'external medicine',
       'breakfast food', 'household general supplis', 'dental care'],
      dtype=object)

In [48]:
mer_mer_cos = get_merchant_similarity_matrix(baskets)

In [49]:
get_most_similar(mer_mer_cos, 357,10)


array([192, 300,  38, 242, 299, 219, 200, 295, 140])

In [50]:
get_most_similar(mer_mer_cos, 357,10)


array([192, 300,  38, 242, 299, 219, 200, 295, 140])

In [51]:
from collections import Counter
popular_cat = Counter(cat
    for transactions in transaction_data
    for cat in transactions)

In [52]:
popular_cat

Counter({'general medicine': 17656,
         'instant food': 16118,
         'powder drink': 11076,
         'sweetend condensed milk': 3793,
         'packaged drink': 13872,
         'ingredients': 19416,
         'sachet drink': 14530,
         'external medicine': 2630,
         'snack': 26146,
         'bath soap': 4176,
         'household general supplis': 3293,
         'bottled water': 119,
         'dental care': 2720,
         'laundry soap': 6594,
         'food materials': 1149,
         'carpentry tools': 168,
         'floor washing soap': 864,
         'medical supplies': 377,
         'packaged tea': 2333,
         'household hygiene supplies': 3557,
         'baby care': 2306,
         'dish soap': 463,
         'hair care': 4129,
         'body care': 3429,
         'canned food': 37,
         'stationary': 388,
         'gas lighter': 8,
         'toys': 119,
         'baby food': 510,
         'womens hygiene supplies': 1674,
         'cream soap': 186,
         'p

In [53]:
popular_cat['general medicine']

17656

In [54]:
popular_cat.most_common()

[('snack', 26146),
 ('ingredients', 19416),
 ('general medicine', 17656),
 ('instant food', 16118),
 ('sachet drink', 14530),
 ('packaged drink', 13872),
 ('powder drink', 11076),
 ('laundry soap', 6594),
 ('bath soap', 4176),
 ('hair care', 4129),
 ('sweetend condensed milk', 3793),
 ('household hygiene supplies', 3557),
 ('body care', 3429),
 ('household general supplis', 3293),
 ('dental care', 2720),
 ('external medicine', 2630),
 ('packaged tea', 2333),
 ('baby care', 2306),
 ('womens hygiene supplies', 1674),
 ('food materials', 1149),
 ('floor washing soap', 864),
 ('baby food', 510),
 ('dish soap', 463),
 ('syrup', 393),
 ('stationary', 388),
 ('medical supplies', 377),
 ('cream soap', 186),
 ('carpentry tools', 168),
 ('bottled water', 119),
 ('toys', 119),
 ('breakfast food', 99),
 ('canned food', 37),
 ('gas lighter', 8),
 ('playing cards', 8)]